# Document Flare Remover

This document correction solution removes flare or glare from document images and provides flare free clear images.

### Prerequisite

To run this algorithm you need to have access to the following AWS Services:
- Access to AWS SageMaker and the model package.
- An S3 bucket to specify input/output.
- Role for AWS SageMaker to access input/output from S3.

This sample notebook shows you how to deploy Synthetic Data Evaluation using Amazon SageMaker.

> **Note**: This is a reference notebook and it cannot run unless you make changes suggested in the notebook.

#### Pre-requisites:
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**
1. To deploy this ML model successfully, ensure that:
    1. Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used: 
        1. **aws-marketplace:ViewSubscriptions**
        1. **aws-marketplace:Unsubscribe**
        1. **aws-marketplace:Subscribe**  
    2. or your AWS account has a subscription to Synthetic Data Evaluation. If so, skip step: [Subscribe to the model package](#1.-Subscribe-to-the-model-package)

#### Contents:
1. [Subscribe to the model package](#1.-Subscribe-to-the-model-package)
2. [Create an endpoint and perform real-time inference](#2.-Create-an-endpoint-and-perform-real-time-inference)
   1. [Create an endpoint](#A.-Create-an-endpoint)
   2. [Create input payload](#B.-Create-input-payload)
   3. [Perform real-time inference](#C.-Perform-real-time-inference)
   4. [Output Result](#D.-Output-Result)
   5. [Delete the endpoint](#E.-Delete-the-endpoint)
3. [Perform batch inference](#3.-Perform-batch-inference) 
4. [Clean-up](#4.-Clean-up)
    1. [Delete the model](#A.-Delete-the-model)
    2. [Unsubscribe to the listing (optional)](#B.-Unsubscribe-to-the-listing-(optional))
    

#### Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

### 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page Synthetic Data Evaluation.
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

In [1]:
model_package_arn='arn:aws:sagemaker:us-east-2:786796469737:model-package/document-flare-removal-copy-05-11'

In [2]:
import pandas as pd
import numpy as np
import os
import json
import boto3
from zipfile import ZipFile
import uuid
import sagemaker as sage
from sagemaker import ModelPackage
from sagemaker import get_execution_role
from IPython.display import Image, display

In [3]:
role = get_execution_role()

sagemaker_session = sage.Session()

bucket = sagemaker_session.default_bucket()
bucket

'sagemaker-us-east-2-786796469737'

### 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

In [4]:
model_name = "flare-removal"

content_type = "application/zip"

real_time_inference_instance_type = "ml.m5.large"
batch_transform_inference_instance_type = "ml.m5.large"

#### A. Create an endpoint

In [5]:
def predict_wrapper(endpoint, session):
    return sage.predictor.RealTimePredictor(endpoint, session, content_type)

In [6]:
#Create a deployable model from model package
model = ModelPackage(role=role,
                    model_package_arn=model_package_arn, 
                    sagemaker_session=sagemaker_session,
                    predictor_cls=predict_wrapper)

In [7]:
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name)

------!

The class RealTimePredictor has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


#### B. Create input payload

#### Instructions

    1) The payload should be in zip format.

In [9]:
file_name = "input.zip"

#### C. Perform real-time inference

In [10]:
!aws sagemaker-runtime invoke-endpoint --endpoint-name $model_name --body fileb://$file_name --content-type 'application/zip' --region us-east-2 output.zip

{
    "ContentType": "application/zip",
    "InvokedProductionVariant": "AllTraffic"
}


#### D. Output Result

- The output file (in zip format) contains the following files:

    1. 'output.zip': A folder containing different high resolution images.

In [11]:
file_path = os.getcwd()
file_name = 'output.zip'

file_object = open(file_name,'rb')
z = ZipFile(file_object)
file_names = []
for name in z.namelist():
    z.extract(name,file_path)
    file_names.append(name)
file_object.close()

output_folder_path = os.path.join(file_path, "output")

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.m

In [12]:
predictor=sage.predictor.Predictor(model_name, sagemaker_session,content_type)

### 3. Perform batch inference

In this section, you will perform batch inference using multiple input payloads together. If you are not familiar with batch transform, and want to learn more, see these links:
1. [How it works](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-batch-transform.html)
2. [How to run a batch transform job](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-batch.html)

In [13]:
#upload the batch-transform job input files to S3
transform_input_folder = "data/input/batch"
transform_input = sagemaker_session.upload_data(transform_input_folder, key_prefix=model_name)
print("Transform input uploaded to " + transform_input)

Transform input uploaded to s3://sagemaker-us-east-2-786796469737/flare-removal


In [14]:
#Run the batch inference job
transformer = model.transformer(1, batch_transform_inference_instance_type)
transformer.transform(transform_input, content_type=content_type)
transformer.wait()

............................... * Serving Flask app 'serve' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on all addresses.
 * Running on http://169.254.255.131:8080/ (Press CTRL+C to quit)
 * Serving Flask app 'serve' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on all addresses.
 * Running on http://169.254.255.131:8080/ (Press CTRL+C to quit)
169.254.255.130 - - [11/May/2023 07:26:58] "GET /ping HTTP/1.1" 200 -
169.254.255.130 - - [11/May/2023 07:26:58] "#033GET /execution-parameters HTTP/1.1#033" 404 -
169.254.255.130 - - [11/May/2023 07:26:58] "GET /ping HTTP/1.1" 200 -
169.254.255.130 - - [11/May/2023 07:26:58] "#033GET /execution-parameters HTTP/1.1#033" 404 -
2023-05-11T07:26:58.752:[sagemaker logs]: MaxConcurrentTransforms=1, MaxPayloadInMB=6, BatchStrategy=MULTI_RECORD
Extracting all the files now...
Imput Image Dir /opt/program/DewarpNet/eval

W0511 07:27:13.845505 140245287114560 util.py:204] Unresolved object in checkpoint: (root).layer_with_weights-20.call_and_return_all_conditional_losses.trace_0
W0511 07:27:13.845577 140245287114560 util.py:204] Unresolved object in checkpoint: (root).layer_with_weights-21.__call__.trace_0
W0511 07:27:13.845644 140245287114560 util.py:204] Unresolved object in checkpoint: (root).layer_with_weights-21.call_and_return_all_conditional_losses.trace_0
W0511 07:27:13.845719 140245287114560 util.py:204] Unresolved object in checkpoint: (root).layer_with_weights-22.__call__.trace_0
W0511 07:27:13.845783 140245287114560 util.py:204] Unresolved object in checkpoint: (root).layer_with_weights-22.call_and_return_all_conditional_losses.trace_0
W0511 07:27:13.845853 140245287114560 util.py:212] A checkpoint was restored (e.g. tf.train.Checkpoint.restore or tf.keras.Model.load_weights) but not all checkpointed values were used. See above for specific issues. Use expect_partial() on the load status obj

W0511 07:27:13.843143 140245287114560 util.py:204] Unresolved object in checkpoint: (root).layer_with_weights-10.call_and_return_all_conditional_losses.trace_0
W0511 07:27:13.843206 140245287114560 util.py:204] Unresolved object in checkpoint: (root).layer-17.__call__.trace_0
W0511 07:27:13.843282 140245287114560 util.py:204] Unresolved object in checkpoint: (root).layer-17.call_and_return_all_conditional_losses.trace_0
W0511 07:27:13.843354 140245287114560 util.py:204] Unresolved object in checkpoint: (root).layer_with_weights-11.__call__.trace_0
W0511 07:27:13.843427 140245287114560 util.py:204] Unresolved object in checkpoint: (root).layer_with_weights-11.call_and_return_all_conditional_losses.trace_0
W0511 07:27:13.843491 140245287114560 util.py:204] Unresolved object in checkpoint: (root).layer_with_weights-12.__call__.trace_0
W0511 07:27:13.843562 140245287114560 util.py:204] Unresolved object in checkpoint: (root).layer_with_weights-12.call_and_return_all_conditional_losses.trac

In [15]:
#Ouput is available on the following path
transformer.output_path

's3://sagemaker-us-east-2-786796469737/document-flare-removal-copy-05-11-2023-05-11-07-21-53-379'

In [16]:
s3_conn = boto3.client("s3")
bucket_name="sagemaker-us-east-2-786796469737"
with open('output.zip', 'wb') as f:
    s3_conn.download_fileobj(bucket_name, os.path.basename(transformer.output_path)+'/input.zip.out', f)
    print("Output file loaded from bucket")

Output file loaded from bucket


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


In [17]:
file_path = os.getcwd()
file_name = 'output.zip'

file_object = open(file_name,'rb')
z = ZipFile(file_object)
file_names = []
for name in z.namelist():
    z.extract(name,file_path)
    file_names.append(name)
file_object.close()

output_folder_path = os.path.join(file_path, "output")

### 4. Clean-up

#### A. Delete the model

In [18]:
predictor.delete_endpoint(delete_endpoint_config=True)
model.delete_model()

#### B. Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.